# Imports

In [1]:
!pip install geopy


[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [42]:
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm
from geopy.distance import geodesic
import time

import warnings
warnings.filterwarnings("ignore")

# Request API from data.gov.sg
Link: https://data.gov.sg/dataset/resale-flat-prices

In [44]:
url = 'https://data.gov.sg/api/action/datastore_search?resource_id=f1765b54-a209-4718-8d38-a39237f502b3&limit=200000'
r = requests.get(url)
data = r.json()
# data

In [45]:
len(data['result']['records'])

149071

In [51]:
df = pd.DataFrame(data['result']['records'])
df.head()

,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,remaining_lease,lease_commence_date,storey_range,_id,block
0,ANG MO KIO,2 ROOM,Improved,44,ANG MO KIO AVE 10,232000,2017-01,61 years 04 months,1979,10 TO 12,1,406
1,ANG MO KIO,3 ROOM,New Generation,67,ANG MO KIO AVE 4,250000,2017-01,60 years 07 months,1978,01 TO 03,2,108
2,ANG MO KIO,3 ROOM,New Generation,67,ANG MO KIO AVE 5,262000,2017-01,62 years 05 months,1980,01 TO 03,3,602
3,ANG MO KIO,3 ROOM,New Generation,68,ANG MO KIO AVE 10,265000,2017-01,62 years 01 month,1980,04 TO 06,4,465
4,ANG MO KIO,3 ROOM,New Generation,67,ANG MO KIO AVE 5,265000,2017-01,62 years 05 months,1980,01 TO 03,5,601


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149071 entries, 0 to 149070
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   town                 149071 non-null  object
 1   flat_type            149071 non-null  object
 2   flat_model           149071 non-null  object
 3   floor_area_sqm       149071 non-null  object
 4   street_name          149071 non-null  object
 5   resale_price         149071 non-null  object
 6   month                149071 non-null  object
 7   remaining_lease      149071 non-null  object
 8   lease_commence_date  149071 non-null  object
 9   storey_range         149071 non-null  object
 10  _id                  149071 non-null  int64 
 11  block                149071 non-null  object
dtypes: int64(1), object(11)
memory usage: 13.6+ MB


In [53]:
df['storey_range'].unique()

array(['10 TO 12', '01 TO 03', '04 TO 06', '07 TO 09', '13 TO 15',
       '19 TO 21', '22 TO 24', '16 TO 18', '34 TO 36', '37 TO 39',
       '49 TO 51', '28 TO 30', '25 TO 27', '40 TO 42', '31 TO 33',
       '46 TO 48', '43 TO 45'], dtype=object)

In [54]:
# Data manipulation
float_cols = ['floor_area_sqm', 'resale_price']
df[float_cols] = df[float_cols].astype(float)

int_cols = ['lease_commence_date']
df[int_cols] = df[int_cols].astype(float)

def get_mths_remaining_lease(lease_str):
    split_lst = lease_str.split(' ')
    if len(split_lst) == 4:
        return int(split_lst[0]) * 12 + int(split_lst[2])
    else:
        return int(split_lst[0]) * 12
    
df['remaining_lease_mth'] = df['remaining_lease'].apply(get_mths_remaining_lease)
df['storey_range_lower'] = df['storey_range'].apply(lambda x: int(x.split(' ')[0]))

df.drop(['remaining_lease'], axis = 1, inplace = True)
df.head()

,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,lease_commence_date,storey_range,_id,block,remaining_lease_mth,storey_range_lower
0,ANG MO KIO,2 ROOM,Improved,44.0,ANG MO KIO AVE 10,232000.0,2017-01,1979.0,10 TO 12,1,406,736,10
1,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 4,250000.0,2017-01,1978.0,01 TO 03,2,108,727,1
2,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,262000.0,2017-01,1980.0,01 TO 03,3,602,749,1
3,ANG MO KIO,3 ROOM,New Generation,68.0,ANG MO KIO AVE 10,265000.0,2017-01,1980.0,04 TO 06,4,465,745,4
4,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,265000.0,2017-01,1980.0,01 TO 03,5,601,749,1


In [55]:
df['address_str'] = df['block'] + " " + df['street_name']

# Feature Engineering

In [56]:
# Get subset to test here first
test_df = df[:100]

## Coordinates

In [57]:
# Download lat long from onemap API
tqdm.pandas()

def get_coordinates(address):
    for i in range(10):
        try:
            req = requests.get('https://developers.onemap.sg/commonapi/search?searchVal='+address+'&returnGeom=Y&getAddrDetails=Y&pageNum=1')
            resultsdict = eval(req.text)
            if len(resultsdict['results']) > 0:
                return (resultsdict['results'][0]['LATITUDE'], resultsdict['results'][0]['LONGITUDE'])
            else:
                return np.nan
            break
        except:
            time.sleep(10)

test_df['lat_lon'] = test_df['address_str'].progress_apply(get_coordinates)

100%|█████████████████████████████████████████| 100/100 [00:09<00:00, 10.19it/s]


## MRT Stations

In [58]:
# Get list of all MRT stations in Singapore
# Excel of MRT stations obtained on 3/3/2023 from https://en.wikipedia.org/wiki/List_of_Singapore_MRT_stations
mrt_df = pd.read_excel('MRT_List.xlsx')
mrt_df['MRT Stations'] = mrt_df['MRT Stations'] + ' MRT Station'
mrt_df

,MRT Stations
0,Admiralty MRT Station
1,Aljunied MRT Station
2,Ang Mo Kio MRT Station
3,Bartley MRT Station
4,Bayfront MRT Station
...,...
141,Woodleigh MRT Station
142,Xilin MRT Station
143,Yew Tee MRT Station
144,Yio Chu Kang MRT Station


In [59]:
# Download lat long from onemap API
tqdm.pandas()

def get_coordinates(address):
    req = requests.get('https://developers.onemap.sg/commonapi/search?searchVal='+address+'&returnGeom=Y&getAddrDetails=Y&pageNum=1')
    resultsdict = eval(req.text)
    if len(resultsdict['results']) > 0:
        return (resultsdict['results'][0]['LATITUDE'], resultsdict['results'][0]['LONGITUDE'])
    else:
        return np.nan

mrt_df['MRT_stn_lat_long'] = mrt_df['MRT Stations'].progress_apply(get_coordinates)

mrt_df = mrt_df.dropna()
mrt_df

100%|█████████████████████████████████████████| 146/146 [00:13<00:00, 10.52it/s]


,MRT Stations,MRT_stn_lat_long
0,Admiralty MRT Station,"(1.44058856161847, 103.800990519771)"
1,Aljunied MRT Station,"(1.3164326118157, 103.882906044385)"
2,Ang Mo Kio MRT Station,"(1.36942855699191, 103.849455226442)"
3,Bartley MRT Station,"(1.34250117805245, 103.880177899184)"
4,Bayfront MRT Station,"(1.28187378879209, 103.859079764874)"
...,...,...
141,Woodleigh MRT Station,"(1.33919004519388, 103.87081830915)"
142,Xilin MRT Station,"(1.32890694938858, 103.964902569595)"
143,Yew Tee MRT Station,"(1.39747594171731, 103.747418249132)"
144,Yio Chu Kang MRT Station,"(1.38168259989517, 103.844991053696)"


In [60]:
def get_shortest_dist_to_mrt(place_coord):
    if pd.isna(place_coord) == True:
        return np.nan
    else:
        shortest = 10e30
        best_mrt = ''

        for idx, row in mrt_df.iterrows():
            mrt = row['MRT Stations']
            dist = geodesic(place_coord, row['MRT_stn_lat_long']).meters
            if dist < shortest:
                shortest = dist
                best_mrt = mrt
        return (shortest, best_mrt)

# test
# get_shortest_dist_to_mrt((1.38070883044887, 103.835368226602))

test_df['closest_mrts'] = test_df['lat_lon'].progress_apply(get_shortest_dist_to_mrt)
test_df.head()

100%|█████████████████████████████████████████| 100/100 [00:02<00:00, 44.55it/s]


,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,lease_commence_date,storey_range,_id,block,remaining_lease_mth,storey_range_lower,address_str,lat_lon,closest_mrts
0,ANG MO KIO,2 ROOM,Improved,44.0,ANG MO KIO AVE 10,232000.0,2017-01,1979.0,10 TO 12,1,406,736,10,406 ANG MO KIO AVE 10,"(1.36200453938712, 103.853879910407)","(957.269782151256, Ang Mo Kio MRT Station)"
1,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 4,250000.0,2017-01,1978.0,01 TO 03,2,108,727,1,108 ANG MO KIO AVE 4,"(1.37094273993861, 103.837974822369)","(166.8282931546725, Mayflower MRT Station)"
2,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,262000.0,2017-01,1980.0,01 TO 03,3,602,749,1,602 ANG MO KIO AVE 5,"(1.38070883044887, 103.835368226602)","(532.1547725841534, Lentor MRT Station)"
3,ANG MO KIO,3 ROOM,New Generation,68.0,ANG MO KIO AVE 10,265000.0,2017-01,1980.0,04 TO 06,4,465,745,4,465 ANG MO KIO AVE 10,"(1.3662010408294, 103.857200967235)","(932.9643884912559, Ang Mo Kio MRT Station)"
4,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,265000.0,2017-01,1980.0,01 TO 03,5,601,749,1,601 ANG MO KIO AVE 5,"(1.38104134784496, 103.835131744823)","(498.4189814658886, Lentor MRT Station)"


In [61]:
def get_num_mrt_1km_radius(place_coord):
    if pd.isna(place_coord) == True:
        return np.nan
    else:
        num = 0
        for idx, row in mrt_df.iterrows():
            mrt = row['MRT Stations']
            dist = geodesic(place_coord, row['MRT_stn_lat_long']).meters
            if dist <= 1000:
                num += 1
        return num

# test
get_num_mrt_1km_radius((1.38070883044887, 103.835368226602))

1

In [62]:
test_df['num_mrts_1km_radius'] = test_df['lat_lon'].progress_apply(get_num_mrt_1km_radius)
test_df.head()

100%|█████████████████████████████████████████| 100/100 [00:02<00:00, 44.97it/s]


,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,lease_commence_date,storey_range,_id,block,remaining_lease_mth,storey_range_lower,address_str,lat_lon,closest_mrts,num_mrts_1km_radius
0,ANG MO KIO,2 ROOM,Improved,44.0,ANG MO KIO AVE 10,232000.0,2017-01,1979.0,10 TO 12,1,406,736,10,406 ANG MO KIO AVE 10,"(1.36200453938712, 103.853879910407)","(957.269782151256, Ang Mo Kio MRT Station)",1
1,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 4,250000.0,2017-01,1978.0,01 TO 03,2,108,727,1,108 ANG MO KIO AVE 4,"(1.37094273993861, 103.837974822369)","(166.8282931546725, Mayflower MRT Station)",2
2,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,262000.0,2017-01,1980.0,01 TO 03,3,602,749,1,602 ANG MO KIO AVE 5,"(1.38070883044887, 103.835368226602)","(532.1547725841534, Lentor MRT Station)",1
3,ANG MO KIO,3 ROOM,New Generation,68.0,ANG MO KIO AVE 10,265000.0,2017-01,1980.0,04 TO 06,4,465,745,4,465 ANG MO KIO AVE 10,"(1.3662010408294, 103.857200967235)","(932.9643884912559, Ang Mo Kio MRT Station)",1
4,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,265000.0,2017-01,1980.0,01 TO 03,5,601,749,1,601 ANG MO KIO AVE 5,"(1.38104134784496, 103.835131744823)","(498.4189814658886, Lentor MRT Station)",1


## Schools

In [63]:
school_df_dic = pd.read_excel('Schools.xlsx', sheet_name = None)

school_df = pd.DataFrame()
for sch_type, df in school_df_dic.items():
    sub_df = pd.DataFrame({'Type': sch_type, 'School': df.iloc[:,0]})
    school_df = pd.concat([school_df, sub_df], ignore_index = True, axis = 0)
    
school_df['lat_long'] = school_df['School'].progress_apply(get_coordinates)
school_df = school_df.dropna()

school_df

100%|█████████████████████████████████████████| 373/373 [00:37<00:00, 10.00it/s]


,Type,School,lat_long
0,Pri,Admiralty Primary School,"(1.4426347903311, 103.800040119743)"
1,Pri,Ahmad Ibrahim Primary School,"(1.43315271543517, 103.832942401086)"
2,Pri,Ai Tong School,"(1.3605834338904, 103.833020333986)"
3,Pri,Alexandra Primary School,"(1.29133439161334, 103.824424680531)"
4,Pri,Anchor Green Primary School,"(1.39036998654612, 103.887165375933)"
...,...,...,...
368,Uni,Nanyang Technological University,"(1.35154257621121, 103.687214243669)"
369,Uni,Singapore Management University,"(1.29509045190873, 103.850566200283)"
370,Uni,Singapore University of Technology and Design,"(1.34078410232498, 103.962542022484)"
371,Uni,Singapore Institute of Technology,"(1.34342966240204, 103.932354333699)"


In [64]:
def get_num_sch_1km_radius(place_coord, sch_type):
    if pd.isna(place_coord) == True:
        return np.nan
    else:
        sub_df = school_df[school_df['Type'] == sch_type]
        num = 0
        for idx, row in sub_df.iterrows():
            dist = geodesic(place_coord, row['lat_long']).meters
            if dist < 1000:
                num += 1
        return num

# test
get_num_sch_1km_radius((1.38070883044887, 103.835368226602), 'Pri')

3

In [65]:
for sch_type in school_df['Type'].unique():
    test_df[f'num_{sch_type}_sch_1km_radius'] = test_df['lat_lon'].progress_apply(lambda x: get_num_sch_1km_radius(x, sch_type))
test_df.head()

100%|████████████████████████████████████████| 100/100 [00:00<00:00, 901.66it/s]


,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,lease_commence_date,storey_range,_id,...,storey_range_lower,address_str,lat_lon,closest_mrts,num_mrts_1km_radius,num_Pri_sch_1km_radius,num_Sec_sch_1km_radius,num_JC_sch_1km_radius,num_MI_sch_1km_radius,num_Uni_sch_1km_radius
0,ANG MO KIO,2 ROOM,Improved,44.0,ANG MO KIO AVE 10,232000.0,2017-01,1979.0,10 TO 12,1,...,10,406 ANG MO KIO AVE 10,"(1.36200453938712, 103.853879910407)","(957.269782151256, Ang Mo Kio MRT Station)",1,2,1,0,0,0
1,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 4,250000.0,2017-01,1978.0,01 TO 03,2,...,1,108 ANG MO KIO AVE 4,"(1.37094273993861, 103.837974822369)","(166.8282931546725, Mayflower MRT Station)",2,3,5,0,0,0
2,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,262000.0,2017-01,1980.0,01 TO 03,3,...,1,602 ANG MO KIO AVE 5,"(1.38070883044887, 103.835368226602)","(532.1547725841534, Lentor MRT Station)",1,3,3,0,0,0
3,ANG MO KIO,3 ROOM,New Generation,68.0,ANG MO KIO AVE 10,265000.0,2017-01,1980.0,04 TO 06,4,...,4,465 ANG MO KIO AVE 10,"(1.3662010408294, 103.857200967235)","(932.9643884912559, Ang Mo Kio MRT Station)",1,2,1,0,0,0
4,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,265000.0,2017-01,1980.0,01 TO 03,5,...,1,601 ANG MO KIO AVE 5,"(1.38104134784496, 103.835131744823)","(498.4189814658886, Lentor MRT Station)",1,3,3,0,0,0


In [68]:
def get_closest_sch_by_type(place_coord, sch_type):
    if pd.isna(place_coord) == True:
        return np.nan
    else:
        sub_df = school_df[school_df['Type'] == sch_type]
        shortest = 10e30
        best_school = ''

        for idx, row in sub_df.iterrows():
            school = row['School']
            dist = geodesic(place_coord, row['lat_long']).meters
            if dist < shortest:
                shortest = dist
                best_school = school
        return (shortest, best_school)

# test
# get_closest_sch_by_type((1.38070883044887, 103.835368226602), 'Pri')

In [69]:
for sch_type in school_df['Type'].unique():
    test_df[f'closest_{sch_type}_sch'] = test_df['lat_lon'].progress_apply(lambda x: get_closest_sch_by_type(x, sch_type))
test_df.head()

100%|████████████████████████████████████████| 100/100 [00:00<00:00, 897.52it/s]


,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,lease_commence_date,storey_range,_id,...,num_Pri_sch_1km_radius,num_Sec_sch_1km_radius,num_JC_sch_1km_radius,num_MI_sch_1km_radius,num_Uni_sch_1km_radius,closest_Pri_sch,closest_Sec_sch,closest_JC_sch,closest_MI_sch,closest_Uni_sch
0,ANG MO KIO,2 ROOM,Improved,44.0,ANG MO KIO AVE 10,232000.0,2017-01,1979.0,10 TO 12,1,...,2,1,0,0,0,"(218.12524012850258, Townsville Primary School)","(465.76727215019685, Deyi Secondary School)","(1401.6125887232145, Eunoia Junior College)","(1779.2162312242424, ITE College Central)","(7408.201434411339, Singapore Management Unive..."
1,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 4,250000.0,2017-01,1978.0,01 TO 03,2,...,3,5,0,0,0,"(257.08353288790374, Ang Mo Kio Primary School)","(414.4895793821761, Mayflower Secondary School)","(1041.0373125705032, Eunoia Junior College)","(1726.0561361616835, Nanyang Polytechnic)","(8371.12577245864, Singapore University of Soc..."
2,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,262000.0,2017-01,1980.0,01 TO 03,3,...,3,3,0,0,0,"(506.84942465370995, Mayflower Primary School)","(809.2416788065827, CHIJ St. Nicholas Girls' S...","(1172.923988792892, Anderson Serangoon Junior ...","(1599.6015685782825, Nanyang Polytechnic)","(8802.805943058956, Singapore University of So..."
3,ANG MO KIO,3 ROOM,New Generation,68.0,ANG MO KIO AVE 10,265000.0,2017-01,1980.0,04 TO 06,4,...,2,1,0,0,0,"(698.1654838332721, Teck Ghee Primary School)","(518.558536057534, Deyi Secondary School)","(1827.2335370609392, Eunoia Junior College)","(1295.7106477607745, ITE College Central)","(7897.637286133394, Singapore Management Unive..."
4,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,265000.0,2017-01,1980.0,01 TO 03,5,...,3,3,0,0,0,"(546.443512815755, Mayflower Primary School)","(842.134462920121, CHIJ St. Nicholas Girls' Sc...","(1205.4131977020443, Anderson Serangoon Junior...","(1625.2990981326832, Nanyang Polytechnic)","(8807.296191921814, Singapore University of So..."


## Malls

In [70]:
malls_df_dic = pd.read_excel('Malls.xlsx', sheet_name = None)

malls_df = pd.DataFrame()
for mall_type, df in malls_df_dic.items():
    sub_df = df.iloc[:,0]
    malls_df = pd.concat([malls_df, sub_df], ignore_index = True, axis = 0)
    
malls_df.columns = ['Malls']
malls_df['lat_long'] = malls_df['Malls'].progress_apply(get_coordinates)
malls_df = malls_df.dropna()

malls_df

100%|█████████████████████████████████████████| 167/167 [00:20<00:00,  8.20it/s]


,Malls,lat_long
0,100 AM,"(1.27468281482263, 103.843488359469)"
1,313@Somerset,"(1.30101436404056, 103.838360664485)"
2,Aperia,"(1.3097112065077, 103.864326436447)"
3,Balestier Hill Shopping Centre,"(1.32559594839311, 103.842571612968)"
4,Bugis Cube,"(1.2981408343975, 103.855635339249)"
...,...,...
162,Gek Poh Shopping Centre,"(1.34874357136408, 103.697732091001)"
163,Rochester Mall,"(1.30540765569962, 103.788446680148)"
164,Taman Jurong Shopping Centre,"(1.33484487471259, 103.720462024278)"
165,West Coast Plaza,"(1.30369748971099, 103.766131294678)"


In [71]:
def get_shortest_dist_to_mall(place_coord):
    if pd.isna(place_coord) == True:
        return np.nan
    else:
        shortest = 10e30
        best_mall = ''

        for idx, row in malls_df.iterrows():
            mall = row['Malls']
            dist = geodesic(place_coord, row['lat_long']).meters
            if dist < shortest:
                shortest = dist
                best_mall = mall
        return (shortest, best_mall)

# test
get_shortest_dist_to_mall((1.38070883044887, 103.835368226602))

(1525.5730377820635, 'Broadway Plaza')

In [72]:
test_df['closest_malls'] = test_df['lat_lon'].progress_apply(get_shortest_dist_to_mall)
test_df.head()

100%|█████████████████████████████████████████| 100/100 [00:02<00:00, 40.07it/s]


,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,lease_commence_date,storey_range,_id,...,num_Sec_sch_1km_radius,num_JC_sch_1km_radius,num_MI_sch_1km_radius,num_Uni_sch_1km_radius,closest_Pri_sch,closest_Sec_sch,closest_JC_sch,closest_MI_sch,closest_Uni_sch,closest_malls
0,ANG MO KIO,2 ROOM,Improved,44.0,ANG MO KIO AVE 10,232000.0,2017-01,1979.0,10 TO 12,1,...,1,0,0,0,"(218.12524012850258, Townsville Primary School)","(465.76727215019685, Deyi Secondary School)","(1401.6125887232145, Eunoia Junior College)","(1779.2162312242424, ITE College Central)","(7408.201434411339, Singapore Management Unive...","(1013.9920581933297, AMK Hub)"
1,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 4,250000.0,2017-01,1978.0,01 TO 03,2,...,5,0,0,0,"(257.08353288790374, Ang Mo Kio Primary School)","(414.4895793821761, Mayflower Secondary School)","(1041.0373125705032, Eunoia Junior College)","(1726.0561361616835, Nanyang Polytechnic)","(8371.12577245864, Singapore University of Soc...","(894.2659750671924, Broadway Plaza)"
2,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,262000.0,2017-01,1980.0,01 TO 03,3,...,3,0,0,0,"(506.84942465370995, Mayflower Primary School)","(809.2416788065827, CHIJ St. Nicholas Girls' S...","(1172.923988792892, Anderson Serangoon Junior ...","(1599.6015685782825, Nanyang Polytechnic)","(8802.805943058956, Singapore University of So...","(1525.5730377820635, Broadway Plaza)"
3,ANG MO KIO,3 ROOM,New Generation,68.0,ANG MO KIO AVE 10,265000.0,2017-01,1980.0,04 TO 06,4,...,1,0,0,0,"(698.1654838332721, Teck Ghee Primary School)","(518.558536057534, Deyi Secondary School)","(1827.2335370609392, Eunoia Junior College)","(1295.7106477607745, ITE College Central)","(7897.637286133394, Singapore Management Unive...","(893.7956343800417, myVillage At Serangoon Gar..."
4,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,265000.0,2017-01,1980.0,01 TO 03,5,...,3,0,0,0,"(546.443512815755, Mayflower Primary School)","(842.134462920121, CHIJ St. Nicholas Girls' Sc...","(1205.4131977020443, Anderson Serangoon Junior...","(1625.2990981326832, Nanyang Polytechnic)","(8807.296191921814, Singapore University of So...","(1569.3061431735507, Broadway Plaza)"


In [73]:
def get_num_malls_1km_radius(place_coord):
    if pd.isna(place_coord) == True:
        return np.nan
    else:
        num = 0
        for idx, row in malls_df.iterrows():
            dist = geodesic(place_coord, row['lat_long']).meters
            if dist < 1000:
                num += 1
        return num

# test
get_num_malls_1km_radius((1.38070883044887, 103.835368226602))

0

In [74]:
test_df['num_malls_1km_radius'] = test_df['lat_lon'].progress_apply(get_num_malls_1km_radius)
test_df.head()

100%|█████████████████████████████████████████| 100/100 [00:02<00:00, 41.12it/s]


,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,lease_commence_date,storey_range,_id,...,num_JC_sch_1km_radius,num_MI_sch_1km_radius,num_Uni_sch_1km_radius,closest_Pri_sch,closest_Sec_sch,closest_JC_sch,closest_MI_sch,closest_Uni_sch,closest_malls,num_malls_1km_radius
0,ANG MO KIO,2 ROOM,Improved,44.0,ANG MO KIO AVE 10,232000.0,2017-01,1979.0,10 TO 12,1,...,0,0,0,"(218.12524012850258, Townsville Primary School)","(465.76727215019685, Deyi Secondary School)","(1401.6125887232145, Eunoia Junior College)","(1779.2162312242424, ITE College Central)","(7408.201434411339, Singapore Management Unive...","(1013.9920581933297, AMK Hub)",0
1,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 4,250000.0,2017-01,1978.0,01 TO 03,2,...,0,0,0,"(257.08353288790374, Ang Mo Kio Primary School)","(414.4895793821761, Mayflower Secondary School)","(1041.0373125705032, Eunoia Junior College)","(1726.0561361616835, Nanyang Polytechnic)","(8371.12577245864, Singapore University of Soc...","(894.2659750671924, Broadway Plaza)",1
2,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,262000.0,2017-01,1980.0,01 TO 03,3,...,0,0,0,"(506.84942465370995, Mayflower Primary School)","(809.2416788065827, CHIJ St. Nicholas Girls' S...","(1172.923988792892, Anderson Serangoon Junior ...","(1599.6015685782825, Nanyang Polytechnic)","(8802.805943058956, Singapore University of So...","(1525.5730377820635, Broadway Plaza)",0
3,ANG MO KIO,3 ROOM,New Generation,68.0,ANG MO KIO AVE 10,265000.0,2017-01,1980.0,04 TO 06,4,...,0,0,0,"(698.1654838332721, Teck Ghee Primary School)","(518.558536057534, Deyi Secondary School)","(1827.2335370609392, Eunoia Junior College)","(1295.7106477607745, ITE College Central)","(7897.637286133394, Singapore Management Unive...","(893.7956343800417, myVillage At Serangoon Gar...",1
4,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,265000.0,2017-01,1980.0,01 TO 03,5,...,0,0,0,"(546.443512815755, Mayflower Primary School)","(842.134462920121, CHIJ St. Nicholas Girls' Sc...","(1205.4131977020443, Anderson Serangoon Junior...","(1625.2990981326832, Nanyang Polytechnic)","(8807.296191921814, Singapore University of So...","(1569.3061431735507, Broadway Plaza)",0


## Parks

In [75]:
parks_df = pd.read_excel('Parks.xlsx')
parks_df = pd.DataFrame({'Parks': parks_df.iloc[:,0]})
parks_df['lat_long'] = parks_df['Parks'].progress_apply(get_coordinates)
parks_df = parks_df.dropna()
parks_df

100%|███████████████████████████████████████████| 73/73 [00:06<00:00, 10.95it/s]


,Parks,lat_long
0,Admiralty Park,"(1.45399059016631, 103.795875774662)"
1,Ang Mo Kio Town Garden East,"(1.37069218596975, 103.850477038504)"
2,Ang Mo Kio Town Garden West,"(1.37416758787361, 103.842948749695)"
3,Bedok Town Park,"(1.33439933330273, 103.922348727352)"
4,Bishan-Ang Mo Kio Park,"(1.36517069591469, 103.83629822663)"
...,...,...
68,Yishun Neighbourhood Park,"(1.43774180535158, 103.835167452882)"
69,Yishun Park,"(1.44144025471558, 103.835602638346)"
70,Yishun Pond Park,"(1.42782773147436, 103.840061845933)"
71,Youth Olympic Park,"(1.28895335576015, 103.860431957375)"


In [76]:
def get_shortest_dist_to_park(place_coord):
    if pd.isna(place_coord) == True:
        return np.nan
    else:
        shortest = 10e30
        best_park = ''

        for idx, row in parks_df.iterrows():
            park = row['Parks']
            dist = geodesic(place_coord, row['lat_long']).meters
            if dist < shortest:
                shortest = dist
                best_park = park
        return (shortest, best_park)

# test
get_shortest_dist_to_park((1.38070883044887, 103.835368226602))

(1111.2380406759219, 'Ang Mo Kio Town Garden West')

In [77]:
test_df['closest_parks'] = test_df['lat_lon'].progress_apply(get_shortest_dist_to_park)
test_df.head()

100%|█████████████████████████████████████████| 100/100 [00:01<00:00, 90.59it/s]


,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,lease_commence_date,storey_range,_id,...,num_MI_sch_1km_radius,num_Uni_sch_1km_radius,closest_Pri_sch,closest_Sec_sch,closest_JC_sch,closest_MI_sch,closest_Uni_sch,closest_malls,num_malls_1km_radius,closest_parks
0,ANG MO KIO,2 ROOM,Improved,44.0,ANG MO KIO AVE 10,232000.0,2017-01,1979.0,10 TO 12,1,...,0,0,"(218.12524012850258, Townsville Primary School)","(465.76727215019685, Deyi Secondary School)","(1401.6125887232145, Eunoia Junior College)","(1779.2162312242424, ITE College Central)","(7408.201434411339, Singapore Management Unive...","(1013.9920581933297, AMK Hub)",0,"(1032.5860223409122, Ang Mo Kio Town Garden East)"
1,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 4,250000.0,2017-01,1978.0,01 TO 03,2,...,0,0,"(257.08353288790374, Ang Mo Kio Primary School)","(414.4895793821761, Mayflower Secondary School)","(1041.0373125705032, Eunoia Junior College)","(1726.0561361616835, Nanyang Polytechnic)","(8371.12577245864, Singapore University of Soc...","(894.2659750671924, Broadway Plaza)",1,"(658.4511966197836, Ang Mo Kio Town Garden West)"
2,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,262000.0,2017-01,1980.0,01 TO 03,3,...,0,0,"(506.84942465370995, Mayflower Primary School)","(809.2416788065827, CHIJ St. Nicholas Girls' S...","(1172.923988792892, Anderson Serangoon Junior ...","(1599.6015685782825, Nanyang Polytechnic)","(8802.805943058956, Singapore University of So...","(1525.5730377820635, Broadway Plaza)",0,"(1111.2380406759219, Ang Mo Kio Town Garden West)"
3,ANG MO KIO,3 ROOM,New Generation,68.0,ANG MO KIO AVE 10,265000.0,2017-01,1980.0,04 TO 06,4,...,0,0,"(698.1654838332721, Teck Ghee Primary School)","(518.558536057534, Deyi Secondary School)","(1827.2335370609392, Eunoia Junior College)","(1295.7106477607745, ITE College Central)","(7897.637286133394, Singapore Management Unive...","(893.7956343800417, myVillage At Serangoon Gar...",1,"(898.0872987909318, Ang Mo Kio Town Garden East)"
4,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,265000.0,2017-01,1980.0,01 TO 03,5,...,0,0,"(546.443512815755, Mayflower Primary School)","(842.134462920121, CHIJ St. Nicholas Girls' Sc...","(1205.4131977020443, Anderson Serangoon Junior...","(1625.2990981326832, Nanyang Polytechnic)","(8807.296191921814, Singapore University of So...","(1569.3061431735507, Broadway Plaza)",0,"(1155.1998420142247, Ang Mo Kio Town Garden West)"


In [78]:
def get_num_parks_1km_radius(place_coord):
    if pd.isna(place_coord) == True:
        return np.nan
    else:
        num = 0
        for idx, row in parks_df.iterrows():
            dist = geodesic(place_coord, row['lat_long']).meters
            if dist < 1000:
                num += 1
        return num

# test
get_num_parks_1km_radius((1.38070883044887, 103.835368226602))

0

In [79]:
test_df['num_parks_1km_radius'] = test_df['lat_lon'].progress_apply(get_num_parks_1km_radius)
test_df.head()

100%|█████████████████████████████████████████| 100/100 [00:01<00:00, 92.83it/s]


,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,lease_commence_date,storey_range,_id,...,num_Uni_sch_1km_radius,closest_Pri_sch,closest_Sec_sch,closest_JC_sch,closest_MI_sch,closest_Uni_sch,closest_malls,num_malls_1km_radius,closest_parks,num_parks_1km_radius
0,ANG MO KIO,2 ROOM,Improved,44.0,ANG MO KIO AVE 10,232000.0,2017-01,1979.0,10 TO 12,1,...,0,"(218.12524012850258, Townsville Primary School)","(465.76727215019685, Deyi Secondary School)","(1401.6125887232145, Eunoia Junior College)","(1779.2162312242424, ITE College Central)","(7408.201434411339, Singapore Management Unive...","(1013.9920581933297, AMK Hub)",0,"(1032.5860223409122, Ang Mo Kio Town Garden East)",0
1,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 4,250000.0,2017-01,1978.0,01 TO 03,2,...,0,"(257.08353288790374, Ang Mo Kio Primary School)","(414.4895793821761, Mayflower Secondary School)","(1041.0373125705032, Eunoia Junior College)","(1726.0561361616835, Nanyang Polytechnic)","(8371.12577245864, Singapore University of Soc...","(894.2659750671924, Broadway Plaza)",1,"(658.4511966197836, Ang Mo Kio Town Garden West)",2
2,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,262000.0,2017-01,1980.0,01 TO 03,3,...,0,"(506.84942465370995, Mayflower Primary School)","(809.2416788065827, CHIJ St. Nicholas Girls' S...","(1172.923988792892, Anderson Serangoon Junior ...","(1599.6015685782825, Nanyang Polytechnic)","(8802.805943058956, Singapore University of So...","(1525.5730377820635, Broadway Plaza)",0,"(1111.2380406759219, Ang Mo Kio Town Garden West)",0
3,ANG MO KIO,3 ROOM,New Generation,68.0,ANG MO KIO AVE 10,265000.0,2017-01,1980.0,04 TO 06,4,...,0,"(698.1654838332721, Teck Ghee Primary School)","(518.558536057534, Deyi Secondary School)","(1827.2335370609392, Eunoia Junior College)","(1295.7106477607745, ITE College Central)","(7897.637286133394, Singapore Management Unive...","(893.7956343800417, myVillage At Serangoon Gar...",1,"(898.0872987909318, Ang Mo Kio Town Garden East)",1
4,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,265000.0,2017-01,1980.0,01 TO 03,5,...,0,"(546.443512815755, Mayflower Primary School)","(842.134462920121, CHIJ St. Nicholas Girls' Sc...","(1205.4131977020443, Anderson Serangoon Junior...","(1625.2990981326832, Nanyang Polytechnic)","(8807.296191921814, Singapore University of So...","(1569.3061431735507, Broadway Plaza)",0,"(1155.1998420142247, Ang Mo Kio Town Garden West)",0


In [80]:
test_df.columns

Index(['town', 'flat_type', 'flat_model', 'floor_area_sqm', 'street_name',
       'resale_price', 'month', 'lease_commence_date', 'storey_range', '_id',
       'block', 'remaining_lease_mth', 'storey_range_lower', 'address_str',
       'lat_lon', 'closest_mrts', 'num_mrts_1km_radius',
       'num_Pri_sch_1km_radius', 'num_Sec_sch_1km_radius',
       'num_JC_sch_1km_radius', 'num_MI_sch_1km_radius',
       'num_Uni_sch_1km_radius', 'closest_Pri_sch', 'closest_Sec_sch',
       'closest_JC_sch', 'closest_MI_sch', 'closest_Uni_sch', 'closest_malls',
       'num_malls_1km_radius', 'closest_parks', 'num_parks_1km_radius'],
      dtype='object')